In [8]:
import cv2
import mediapipe as mp
import os
import pandas as pd
import tqdm as tq

In [9]:
MODEL = 1  # 1 = Lite model, 2 = Full model
video_path = "../Videos/5003_I.MOV"
video_extensions = ["*.mp4", "*.avi", "*.MOV"]

if os.path.exists(video_path) == True:
    print(f"{video_path} is a valid file. Proceed with processing.")
else:
    raise ValueError(f"{video_path} does not exist. Try adding the entire file path.")

../Videos/5003_I.MOV is a valid file. Proceed with processing.


In [10]:
# Initialize MediaPipe components
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [11]:
data = []
with mp_holistic.Holistic(static_image_mode=False, model_complexity=MODEL, 
                          min_detection_confidence=0.4, min_tracking_confidence=0.4) as holistic:
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Get the frame width and height from the video capture object
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # # Define the codec using VideoWriter_fourcc and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, fps, (frame_width, frame_height), isColor=True)
    
    for _ in tq.tqdm(range(total_frames), desc="Processing frames", unit="frame"):
        ret, image = cap.read()
        if not ret:
            break
        
        frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES)
        time_ms = round((frame_number / fps) * 1000, 0)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        
        # Convert the image color back to BGR for VideoWriter
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw the pose landmarks on the image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        # # Write the frame into the file 'output.avi'
        out.write(image)
        
        if results.pose_landmarks is not None:
            right_shoulder_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x
            right_shoulder_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y
            left_shoulder_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x
            left_shoulder_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
            right_elbow_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x
            right_elbow_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
            left_elbow_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x
            left_elbow_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
            right_wrist_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x
            right_wrist_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
            left_wrist_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x
            left_wrist_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
            right_eye_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].x
            right_eye_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].y
            left_eye_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].x
            left_eye_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].y
            nose_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x
            nose_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y
            right_index_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].x
            right_index_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].y

            data.append([time_ms, right_shoulder_x, right_shoulder_y, left_shoulder_x, left_shoulder_y, 
                        right_elbow_x, right_elbow_y, left_elbow_x, left_elbow_y, 
                        right_wrist_x, right_wrist_y, left_wrist_x, left_wrist_y, 
                        right_eye_x, right_eye_y, left_eye_x, left_eye_y, right_index_x, right_index_y,
                        nose_x, nose_y])
            
                    
        out.write(image)

    cap.release()
    out.release()

Processing frames: 100%|██████████| 27147/27147 [15:48<00:00, 28.63frame/s]


In [12]:
# Dictionary of all keypoints, appended to final dataframe
df = pd.DataFrame(data, columns=[
    "time_ms", 
    "right_shoulder_x", "right_shoulder_y", 
    "left_shoulder_x", "left_shoulder_y", 
    "right_elbow_x", "right_elbow_y", 
    "left_elbow_x", "left_elbow_y", 
    "right_wrist_x", "right_wrist_y", 
    "left_wrist_x", "left_wrist_y", 
    "right_eye_x", "right_eye_y", 
    "left_eye_x", "left_eye_y",
    "nose_x", "nose_y",
    "right_index_x", "right_index_y" 
    ])

In [13]:
# Save DataFrame as CSV file
csv_file_name = "../Keypoints/" + os.path.splitext(os.path.basename(video_path))[0] + "_keypoints.csv"
df.to_csv(csv_file_name, index=False)
print(f"DataFrame saved as {csv_file_name}")

DataFrame saved as ../Keypoints/5003_I_keypoints.csv
